In [213]:
import math
import cmath
import altair as alt
import numpy as np
import pandas as pd

In [224]:
names = ['Freq', 'dB', 'Phase']
speaker = 'Genelec 8361 + W371 GLM (ON)'
left = pd.read_csv("../datas/inroom/m002/L.txt", skiprows=14, names=names)
right = pd.read_csv("../datas/inroom/m002/R.txt", skiprows=14, names=names)
# left.Phase /= 180/math.pi
# right.Phase /= 180/math.pi

In [225]:
def minimum_phase(df):
    x = df.dB.values
    n = len(x)
    spectrum = np.fft.fft(x, n=n)
    ceps = np.fft.ifft(np.log(np.abs(spectrum))).real
    odd = n % 2
    window = np.concatenate(([1.0], 2.0 * np.ones(int((n + odd) / 2 - 1)), np.ones(1 - odd), np.zeros(int((n + odd) / 2 - 1))))
    m = np.fft.ifft(np.exp(np.fft.fft(window * ceps))).real
    print('Min Phase [{}:{}]'.format(np.min(m), np.max(m)))
    return m

left['MinPhase'] = minimum_phase(left)
left['ExcessPhase'] = np.angle( np.array([cmath.rect(1, l) for l in left.Phase])-np.array([cmath.rect(1, m) for m in left.MinPhase]), deg=True)
right['MinPhase'] = minimum_phase(right)
right['ExcessPhase'] = np.angle( np.array([cmath.rect(1, l) for l in right.Phase])-np.array([cmath.rect(1, m) for m in right.MinPhase]), deg=True)

Min Phase [55.45587114176065:91.72578564531298]
Min Phase [56.05898140220235:95.02123829792649]


In [226]:
def plot(df, f_max):
    x_axis=alt.X('Freq:Q', title='Frequency (Hz)', axis=alt.Axis(format='s'), scale=alt.Scale(type="log", nice=False, domain=[20, f_max]))
    freq = alt.Chart(df.loc[df.Freq<=f_max]).mark_line(
    ).encode(
        x=x_axis,
        y=alt.Y('dB:Q', title='Sound Pressure (dB SPL)', scale=alt.Scale(zero=False, domain=[30, 80]))
    ).properties(width=500, height=300)
    phase = alt.Chart(df.loc[df.Freq<=f_max]).mark_line(
    ).encode(
        x=x_axis,
        y=alt.Y('Phase:Q', title='Phase (deg)', scale=alt.Scale(zero=False, nice=False, domain=[-180, 180])),
        color=alt.value('green')
    ).properties(width=500, height=200)
    min_phase = alt.Chart(df.loc[df.Freq<=f_max]).mark_line(
    ).encode(
        x=x_axis,
        y=alt.Y('MinPhase:Q', title='Phase (deg)', scale=alt.Scale(zero=False, nice=False, domain=[-180, 180])),
        color=alt.value('red')
    ).properties(width=500, height=200)
    excess_phase = alt.Chart(df.loc[df.Freq<=f_max]
    ).mark_line(
    ).encode(
        x=x_axis,
        y=alt.Y('ExcessPhase:Q', title='Phase (deg)', scale=alt.Scale(zero=False, nice=False, domain=[-180, 180])),
        color=alt.value('yellow')
    ).properties(width=500, height=200)
    chart = alt.vconcat(freq, phase+min_phase+excess_phase)
    return chart

In [227]:
f_max = 200
alt.hconcat(
    plot(left, f_max).properties(title='{} Left'.format(speaker)), 
    plot(right, f_max).properties(title='{} Right'.format(speaker)))

alt.HConcatChart(...)